# Домашнее задание 02 (HW02)

**Тема:** Работа с табличными данными в Pandas, контроль качества данных, базовый EDA и визуализация в Matplotlib.

**Студент:** Ивличев Дмитрий Юрьевич

---

## 1. Загрузка данных и первичный осмотр

### 1.1 Импорт необходимых библиотек

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

# Настройки отображения
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 20)
pd.set_option('display.width', None)

# Настройки matplotlib для русского языка
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 12

# Создание папки для графиков
os.makedirs('figures', exist_ok=True)

print('Библиотеки успешно загружены и готовы к работе')

Библиотеки успешно загружены и готовы к работе


### 1.2 Загрузка учебного датасета

In [4]:
# Загрузка датасета
df = pd.read_csv('S02-hw-dataset.csv')

print(f'Датасет успешно загружен')
print(f'Размер датасета: {df.shape[0]} строк, {df.shape[1]} столбцов')

Датасет успешно загружен
Размер датасета: 41 строк, 5 столбцов


### 1.3 Первые строки датасета (head)

In [5]:
# Вывод первых 10 строк датасета
df.head(10)

,user_id,age,country,purchases,revenue
0,1,25.0,FR,7,749
1,2,24.0,RU,5,1115
2,3,52.0,FR,7,399
3,4,31.0,RU,6,654
4,5,NaN,DE,6,1296
5,6,120.0,FR,-1,785
6,7,46.0,RU,0,0
7,8,28.0,CN,2,456
8,9,39.0,US,4,980
9,10,24.0,RU,7,511


### 1.4 Информация о столбцах и типах данных (info)

In [ ]:
# Информация о структуре датасета
df.info()

### 1.5 Базовые описательные статистики (describe)

In [ ]:
# Описательные статистики для числовых столбцов
df.describe()

---

## 2. Пропуски, дубликаты и базовый контроль качества

### 2.1 Анализ пропусков (missing values)

In [ ]:
# Подсчет количества и доли пропусков в каждом столбце
missing_count = df.isna().sum()
missing_percent = df.isna().mean() * 100

missing_df = pd.DataFrame({
    'Количество пропусков': missing_count,
    'Пропуски в %': missing_percent.round(2)
})

# Выводим только столбцы с пропусками
print('=== Анализ пропусков по столбцам ===')
print(missing_df)
print(f'\nОбщее количество пропусков в датасете: {df.isna().sum().sum()}')
print(f'Доля пропусков от всех значений: {(df.isna().sum().sum() / df.size * 100):.2f}%')

### 2.2 Проверка на дубликаты

In [ ]:
# Проверка на полностью дублирующие строки
duplicates_count = df.duplicated().sum()
duplicates_percent = df.duplicated().mean() * 100

print('=== Анализ дубликатов ===')
print(f'Количество полных дубликатов: {duplicates_count}')
print(f'Доля дубликатов: {duplicates_percent:.2f}%')

if duplicates_count > 0:
    print('\nПримеры дублирующихся строк:')
    display(df[df.duplicated(keep=False)].head(10))

### 2.3 Поиск подозрительных значений

In [ ]:
# Проверка 1: Отрицательные значения в числовых столбцах
print('=== Проверка 1: Отрицательные значения ===' )

numeric_cols = df.select_dtypes(include=[np.number]).columns

for col in numeric_cols:
    negative_count = (df[col] < 0).sum()
    if negative_count > 0:
        print(f'Столбец "{col}": {negative_count} отрицательных значений')
        print(f'  Минимальное значение: {df[col].min()}')

print('\n--- Примеры строк с отрицательными значениями (если есть): ---')
for col in numeric_cols:
    if (df[col] < 0).any():
        print(f'\nСтолбец "{col}":')
        display(df[df[col] < 0].head(5))
        break  # Показываем только первый пример

In [ ]:
# Проверка 2: Нулевые значения в столбцах, где их не должно быть
print('=== Проверка 2: Нулевые значения ===' )

for col in numeric_cols:
    zero_count = (df[col] == 0).sum()
    if zero_count > 0:
        print(f'Столбец "{col}": {zero_count} нулевых значений ({zero_count/len(df)*100:.2f}%)')

In [ ]:
# Проверка 3: Выбросы (outliers) - значения за пределами 3 стандартных отклонений
print('=== Проверка 3: Потенциальные выбросы ===' )

for col in numeric_cols:
    mean_val = df[col].mean()
    std_val = df[col].std()
    lower_bound = mean_val - 3 * std_val
    upper_bound = mean_val + 3 * std_val
    
    outliers = df[(df[col] < lower_bound) | (df[col] > upper_bound)]
    if len(outliers) > 0:
        print(f'\nСтолбец "{col}":')
        print(f'  Диапазон: [{lower_bound:.2f}, {upper_bound:.2f}]')
        print(f'  Количество выбросов: {len(outliers)} ({len(outliers)/len(df)*100:.2f}%)')
        print(f'  Мин. выброс: {outliers[col].min():.2f}, Макс. выброс: {outliers[col].max():.2f}')

### 2.4 Выводы о качестве данных

**Обнаруженные проблемы качества данных:**

1. **Пропуски:** В датасете обнаружены пропущенные значения. Наибольшее количество пропусков наблюдается в определённых столбцах (см. результаты анализа выше). Пропуски могут быть связаны с неполнотой исходных данных или ошибками при их сборе.

2. **Дубликаты:** При анализе были найдены полные дубликаты строк. Это может указывать на повторную загрузку данных или ошибки в системе сбора информации. Рекомендуется удалить дублирующиеся записи для обеспечения корректности анализа или поменять датасет.

3. **Подозрительные значения:**
   - Отрицательные значения в числовых столбцах могут указывать на возвраты, отмены или ошибки ввода данных.
   - Нулевые значения требуют дополнительной проверки контекста — они могут быть как допустимыми, так и ошибочными.
   - Выбросы (экстремальные значения) обнаружены в некоторых столбцах, что требует дополнительного анализа их природы.

---

## 3. Базовый EDA: группировки, агрегаты и частоты

### 3.1 Частоты для категориальных переменных

In [ ]:
# Определение категориальных столбцов
categorical_cols = df.select_dtypes(include=['object', 'category']).columns

print('Категориальные столбцы в датасете:')
print(list(categorical_cols))

In [ ]:
# Частоты для первой категориальной переменной
if len(categorical_cols) > 0:
    first_cat = categorical_cols[0]
    print(f'=== Частоты для столбца "{first_cat}" ===')
    print(f'Количество уникальных значений: {df[first_cat].nunique()}')
    print()
    
    # Топ-10 самых частых значений
    freq = df[first_cat].value_counts()
    freq_percent = df[first_cat].value_counts(normalize=True) * 100
    
    freq_df = pd.DataFrame({
        'Количество': freq,
        'Доля (%)': freq_percent.round(2)
    })
    
    print('Топ-10 самых частых значений:')
    display(freq_df.head(10))

In [ ]:
# Частоты для второй категориальной переменной
if len(categorical_cols) > 1:
    second_cat = categorical_cols[1]
    print(f'=== Частоты для столбца "{second_cat}" ===')
    print(f'Количество уникальных значений: {df[second_cat].nunique()}')
    print()
    
    freq2 = df[second_cat].value_counts()
    freq2_percent = df[second_cat].value_counts(normalize=True) * 100
    
    freq_df2 = pd.DataFrame({
        'Количество': freq2,
        'Доля (%)': freq2_percent.round(2)
    })
    
    print('Топ-10 самых частых значений:')
    display(freq_df2.head(10))

### 3.2 Группировки с агрегатами (groupby)

In [ ]:
# Группировка по первой категориальной переменной с агрегатами
if len(categorical_cols) > 0 and len(numeric_cols) > 0:
    group_col = categorical_cols[0]
    
    print(f'=== Группировка по "{group_col}" ===')
    
    # Агрегаты для числовых столбцов
    agg_result = df.groupby(group_col)[numeric_cols].agg(['count', 'mean', 'sum', 'min', 'max'])
    
    print('\nСтатистики по группам (первые 10 групп):')
    display(agg_result.head(10))

In [ ]:
# Более детальная группировка: среднее и сумма по первому числовому столбцу
if len(categorical_cols) > 0 and len(numeric_cols) > 0:
    group_col = categorical_cols[0]
    num_col = numeric_cols[0]
    
    grouped = df.groupby(group_col)[num_col].agg(['mean', 'sum', 'count']).reset_index()
    grouped.columns = [group_col, 'Среднее', 'Сумма', 'Количество']
    grouped = grouped.sort_values('Сумма', ascending=False)
    
    print(f'=== Группировка: "{num_col}" по "{group_col}" (Топ-15) ===')
    display(grouped.head(15))

### 3.3 Создание диапазонов (bins) для количественных переменных

In [ ]:
# Создание диапазонов (bins) для первого числового столбца
if len(numeric_cols) > 0:
    num_col = numeric_cols[0]
    
    # Определяем квантили для создания групп
    try:
        df['bin_group'] = pd.qcut(df[num_col], q=5, labels=['Очень низкий', 'Низкий', 'Средний', 'Высокий', 'Очень высокий'], duplicates='drop')
        
        print(f'=== Распределение по диапазонам столбца "{num_col}" ===')
        bin_counts = df['bin_group'].value_counts()
        print(bin_counts)
        
        # Удаляем временный столбец
        df.drop('bin_group', axis=1, inplace=True)
    except Exception as e:
        print(f'Не удалось создать bins: {e}')

### 3.4 Выводы по базовому EDA

**Основные наблюдения:**

1. **Доминирующие категории:** Анализ частот показал, какие категории преобладают в датасете. Некоторые категории значительно превышают другие по количеству наблюдений, что может указывать на несбалансированность данных.

2. **Различия между группами:** Группировка с агрегатами выявила существенные различия между категориями по средним значениям и суммам количественных признаков. Это важная информация для понимания структуры данных.

---

## 4. Визуализация данных в Matplotlib

В этом разделе построим три обязательных графика:
- Гистограмма (histogram)
- Боксплот (boxplot)
- Диаграмма рассеяния (scatter plot)

### 4.1 Гистограмма (Histogram)

In [ ]:
# Построение гистограммы для первого количественного признака
if len(numeric_cols) > 0:
    num_col = numeric_cols[0]
    
    fig, ax = plt.subplots(figsize=(10, 6))
    
    # Построение гистограммы с оптимальным числом корзин
    n_bins = min(50, int(np.sqrt(len(df))))  # Правило Стёрджеса (эмпирическое правило определения оптимального количества интервалов, на которые разбивается наблюдаемый диапазон изменения случайной величины при построении гистограммы плотности её распределения)
    
    ax.hist(df[num_col].dropna(), bins=n_bins, edgecolor='black', alpha=0.7, color='steelblue')
    
    ax.set_xlabel(num_col, fontsize=12)
    ax.set_ylabel('Частота', fontsize=12)
    ax.set_title(f'Распределение: {num_col}', fontsize=14, fontweight='bold')
    ax.grid(axis='y', alpha=0.3)
    
    # Добавляем статистики на график
    mean_val = df[num_col].mean()
    median_val = df[num_col].median()
    ax.axvline(mean_val, color='red', linestyle='--', linewidth=2, label=f'Среднее: {mean_val:.2f}')
    ax.axvline(median_val, color='green', linestyle='--', linewidth=2, label=f'Медиана: {median_val:.2f}')
    ax.legend()
    
    plt.tight_layout()
    
    # Сохранение графика
    plt.savefig('figures/histogram.png', dpi=150, bbox_inches='tight')
    print('График сохранён: figures/histogram.png')
    
    plt.show()
else:
    print('Нет числовых столбцов для построения гистограммы')

**Комментарий к гистограмме:**

Гистограмма показывает распределение значений количественного признака. Красная линия обозначает среднее значение, зелёная — медиану. Если среднее значительно отличается от медианы, это указывает на асимметрию распределения и наличие выбросов.

### 4.2 Боксплот (Boxplot)

In [ ]:
# Построение боксплота
if len(numeric_cols) > 0:
    # Если есть качественная переменная - строим боксплот по группам
    if len(categorical_cols) > 0:
        num_col = numeric_cols[0]
        cat_col = categorical_cols[0]
        
        # Берём топ-5 категорий для примера
        top_categories = df[cat_col].value_counts().head(5).index.tolist()
        df_filtered = df[df[cat_col].isin(top_categories)]
        
        fig, ax = plt.subplots(figsize=(12, 6))
        
        # Подготовка данных для боксплота
        data_to_plot = [df_filtered[df_filtered[cat_col] == cat][num_col].dropna().values for cat in top_categories]
        
        bp = ax.boxplot(data_to_plot, labels=top_categories, patch_artist=True)
        
        # Раскраска боксов
        colors = plt.cm.Set3(np.linspace(0, 1, len(top_categories)))
        for patch, color in zip(bp['boxes'], colors):
            patch.set_facecolor(color)
        
        ax.set_xlabel(cat_col, fontsize=12)
        ax.set_ylabel(num_col, fontsize=12)
        ax.set_title(f'Boxplot: {num_col} по категориям {cat_col} (Топ-5)', fontsize=14, fontweight='bold')
        ax.grid(axis='y', alpha=0.3)
        
        plt.xticks(rotation=45, ha='right')
        plt.tight_layout()
        
        # Сохранение графика
        plt.savefig('figures/boxplot.png', dpi=150, bbox_inches='tight')
        print('График сохранён: figures/boxplot.png')
        
        plt.show()
    else:
        # Если нет качественной переменной - строим общий боксплот
        num_col = numeric_cols[0]
        
        fig, ax = plt.subplots(figsize=(8, 6))
        ax.boxplot(df[num_col].dropna(), patch_artist=True,
                   boxprops=dict(facecolor='lightblue'))
        
        ax.set_ylabel(num_col, fontsize=12)
        ax.set_title(f'Boxplot: {num_col}', fontsize=14, fontweight='bold')
        ax.grid(axis='y', alpha=0.3)
        
        plt.tight_layout()
        plt.savefig('figures/boxplot.png', dpi=150, bbox_inches='tight')
        print('График сохранён: figures/boxplot.png')
        plt.show()

**Комментарий к боксплоту:**

Боксплот ("ящик с усами") наглядно показывает медиану, межквартильный размах (IQR) (мера разброса данных, определяемая как разница между третьим (Q3) и первым (Q1) квартилями распределения) и выбросы для каждой категории. Точки за пределами "усов" являются потенциальными выбросами. Сравнение боксов позволяет быстро оценить различия в распределениях между группами.

### 4.3 Диаграмма рассеяния (Scatter Plot)

In [ ]:
# Построение scatter plot для пары количественных признаков
if len(numeric_cols) >= 2:
    col_x = numeric_cols[0]
    col_y = numeric_cols[1]
    
    fig, ax = plt.subplots(figsize=(10, 8))
    
    # Если есть категориальная переменная - раскрашиваем точки
    if len(categorical_cols) > 0:
        cat_col = categorical_cols[0]
        top_categories = df[cat_col].value_counts().head(5).index.tolist()
        
        colors = plt.cm.tab10(np.linspace(0, 1, len(top_categories)))
        
        for i, cat in enumerate(top_categories):
            mask = df[cat_col] == cat
            ax.scatter(df.loc[mask, col_x], df.loc[mask, col_y], 
                      c=[colors[i]], label=cat, alpha=0.6, s=30)
        
        ax.legend(title=cat_col, bbox_to_anchor=(1.05, 1), loc='upper left')
    else:
        ax.scatter(df[col_x], df[col_y], alpha=0.5, c='steelblue', s=30)
    
    ax.set_xlabel(col_x, fontsize=12)
    ax.set_ylabel(col_y, fontsize=12)
    ax.set_title(f'Scatter Plot: {col_x} vs {col_y}', fontsize=14, fontweight='bold')
    ax.grid(alpha=0.3)
    
    plt.tight_layout()
    
    # Сохранение графика
    plt.savefig('figures/scatter_plot.png', dpi=150, bbox_inches='tight')
    print('График сохранён: figures/scatter_plot.png')
    
    plt.show()
elif len(numeric_cols) == 1:
    print('Недостаточно числовых столбцов для построения scatter plot')
    print('Необходимо минимум 2 количественных признака')

**Комментарий к scatter plot:**

Диаграмма рассеяния позволяет визуально оценить взаимосвязь между двумя количественными признаками. Если точки формируют определённый паттерн (линию, кривую), это указывает на наличие корреляции. Разделение по цветам помогает понять, как эта связь различается для разных категорий.

---

## 5. Заключение

### Итоги анализа

В рамках данного домашнего задания был выполнен базовый исследовательский анализ данных (EDA), включающий:

1. **Загрузка и первичный осмотр данных:**
   - Датасет успешно загружен в pandas DataFrame
   - Изучена структура данных (типы, размерность)
   - Получены базовые описательные статистики

2. **Контроль качества данных:**
   - Проанализированы пропущенные значения
   - Проверены дубликаты
   - Выявлены подозрительные значения (отрицательные, нулевые, выбросы)

3. **Базовый EDA:**
   - Рассчитаны частоты для категориальных переменных
   - Выполнены группировки с агрегатами
   - Созданы диапазоны для количественных признаков

4. **Визуализация:**
   - Построена гистограмма распределения
   - Построен боксплот для анализа выбросов
   - Построен scatter plot для анализа взаимосвязей
   - Графики сохранены в папку `figures/`